In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline




/home/alice/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-14 16:48:16.727154: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 16:48:17.353612: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
sentences = ["Quando mi ha detto che il gatto era morto  mi sono messa a pianegere", "Oggi è una bellissima giornata", "Mi fai troppo arrabbiare quando mi dici così "]

In [10]:

tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(sentences)

[{'label': '0', 'score': 0.778736412525177},
 {'label': '1', 'score': 0.7228806614875793},
 {'label': '3', 'score': 0.8531340956687927}]


    0: sadness
    1: joy
    2: love
    3: anger
    4: fear
    5: surprise


In [3]:


tokenizer = AutoTokenizer.from_pretrained('MilaNLProc/feel-it-italian-emotion')
model = AutoModelForSequenceClassification.from_pretrained('MilaNLProc/feel-it-italian-emotion')

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
out=emotion_analysis(sentences[0],top_k=4)

In [4]:
out

[{'label': 'sadness', 'score': 0.9951210618019104},
 {'label': 'fear', 'score': 0.003921428229659796},
 {'label': 'joy', 'score': 0.0007393912528641522},
 {'label': 'anger', 'score': 0.00021813683270011097}]

In [7]:
for o in out:
    print(o["label"])

sadness
fear
joy
anger


In [1]:
#run the planner
import os
import subprocess
def planning(command,domain_path,plan_file):
    #plan the first time and get the list of action
    os.chdir (domain_path)
    #run the planner
    fd_process = subprocess.Popen([command], stdout=subprocess.PIPE, shell=True)
    try:
        (out, err) = fd_process.communicate()
        f = open(plan_file, "w")
        f.write(out.decode())
        f.close()
        fd_exit = fd_process.returncode
        #print(fd_exit)
        return fd_exit
    except:
        print("exrrorrr")

In [20]:
command="./ff -p /home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg/ -o goal1_domain.pddl -f prova_problem.pddl"
domain_path="/home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg"

In [21]:
os.chdir (domain_path)
#run the planner
fd_process = subprocess.Popen([command], stdout=subprocess.PIPE, shell=True)

In [22]:
(out, err) = fd_process.communicate()


In [24]:
type(a)

str

In [25]:
f = open("plan.pddl", "w")
f.write(a)
f.close()

In [26]:
fd_exit = fd_process.returncode
#print(fd_exit)
fd_exit

0

In [32]:
output_path="/home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg/plan.pddl"

In [43]:
if os.path.isfile(output_path):
    with open(output_path, "r") as plan_file:
        raw_plan = plan_file.readlines()
    plan=[]
    start=False
    for p in raw_plan:
        if "step" in p:
             start=True

        if "time spent" in p:
             start=False
        if start:
            if p[0] !=":":
                plan.append(p)
    actions_to_execute=[]
    for i in plan:
        a=i[i.find(":")+1:i.find("\n")].replace(" ","")
        if a!="":
            actions_to_execute.append(a)   
    return actions_to_execute 

else:
        print("Plan not found")

In [46]:
plan

['step    0: GREETINGS\n',
 '        1: INTRO_ACTION\n',
 '        2: FEELINGS_SKIP\n',
 '        3: CHECK_FINISH\n',
 '     \n',
 '\n']

In [55]:
actions_to_execute

['GREETINGS', 'INTRO_ACTION', 'FEELINGS_SKIP', 'CHECK_FINISH']

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from proper_lpg.load_ontology import *

In [2]:
import rospy

In [3]:
#define the actual personality
traits=["Extrovert","Introvert","Conscientious","Unscrupulous","Agreeable","Disagreeable"]
traits_preds=["(extro)","(intro)","(consc)","(unsc)","(agree)","(disagree)"]
we=0
wi=0.5
wc=0
wu=0
wa=0
wd=0
sum_weights=we +wi +wc + wu + wa + wd
weights=[we/sum_weights,wi/sum_weights,wc/sum_weights,wu/sum_weights,wa/sum_weights,wd/sum_weights]
gamma=1
perception=""
new_perception=False

In [4]:
domain_path='/home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg/domain_prova.pddl'
problem_path='/home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg/prova_problem.pddl'
init_pb='/home/alice/catkin_ws/src/PROPER_Sofar/proper_lpg/init_problem.pddl'

In [5]:
rospy.loginfo('Executing state INIT')
rospy.loginfo('copy pb in the correct file')        
with open(init_pb,'r') as firstfile, open(problem_path,'w') as secondfile:
    for line in firstfile:
    
        if "extroversion_coefficient" in line:
            if we!=0:
                l="        (= (extroversion_coefficient) "+str(gamma*(we/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[0]+"\n"
                secondfile.write(p)
            else:
                l="        (= (extroversion_coefficient) "+str(gamma*(wi/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[1]+"\n"
                secondfile.write(p)
        elif "conscientious_coefficient" in line:
            if wc!=0:
                l="        (= (conscientious_coefficient) "+str(gamma*(wc/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[2]+"\n"
                secondfile.write(p)
            else:
                l="        (= (conscientious_coefficient) "+str(gamma*(wu/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[3]+"\n"
                secondfile.write(p)
        elif "agreeableness_coefficient" in line:
            if wa!=0:
                l="        (= (agreeableness_coefficient) "+str(gamma*(wa/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[4]+"\n"
                secondfile.write(p)
            else:
                l="        (= (agreeableness_coefficient) "+str(gamma*(wd/sum_weights)) +")\n"
                secondfile.write(l)
                p="        "+traits_preds[5]+"\n"
                secondfile.write(p)
        else:
            secondfile.write(line)


In [6]:
rospy.loginfo('Reading domain and populate ontology')
populate_ontology(domain_path)


first ['EXTRO_ACTION', 'INTRO_ACTION', 'CONSC_ACTION', 'UNSC_ACTION', 'AGREE_ACTION', 'DISAGREE_ACTION', 'NO_REACT_SAD_EMOTION', 'REACT_SAD_EMOTION_NEG', 'NO_REACT_ANGER_EMOTION', 'REACT_ANGER_EMOTION_POS', 'REACT_ANGER_EMOTION_NEG', 'NO_REACT_HAPPY_EMOTION', 'REACT_HAPPY_EMOTION_NEG', 'REACT_HAPPY_EMOTION_AGREE', 'REACT_HAPPY_EMOTION_EXTRO', 'NO_REACT_NEUTRAL_EMOTION', 'REACT_NEUTRAL_EMOTION_POS', 'REACT_NEUTRAL_EMOTION_NEG', 'NO_REACT_TOUCH', 'APPROACH_TOUCH_AGREE', 'APPROACH_TOUCH_EXTRO', 'AVOID_TOUCH_INTRO', 'AVOID_TOUCH_DISAGREE', 'GREETINGS', 'FEELINGS', 'CHECK_FINISH']
second ['INTRO_ACTION', 'CONSC_ACTION', 'UNSC_ACTION', 'AGREE_ACTION', 'DISAGREE_ACTION', 'NO_REACT_SAD_EMOTION', 'REACT_SAD_EMOTION_NEG', 'NO_REACT_ANGER_EMOTION', 'REACT_ANGER_EMOTION_POS', 'REACT_ANGER_EMOTION_NEG', 'NO_REACT_HAPPY_EMOTION', 'REACT_HAPPY_EMOTION_NEG', 'REACT_HAPPY_EMOTION_AGREE', 'REACT_HAPPY_EMOTION_EXTRO', 'NO_REACT_NEUTRAL_EMOTION', 'REACT_NEUTRAL_EMOTION_POS', 'REACT_NEUTRAL_EMOTION_NEG', '

KeyError: 'EXTRO_ACTION'

In [72]:
rospy.loginfo('Initialize function and predicates in the ontology')
initialize_functions_predicates()
rospy.loginfo('Read the problem and set the initial values of predicates and functions')
read_the_problem(problem_path) 